In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run imports_notebook.py

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = GPT2LMHeadModel.from_pretrained("distilgpt2")
model = model.to(device)

In [4]:
with open('murder.txt') as f:
    text_book = f.read()
    text_book = ' '.join(text_book.split())
    tokens_str_book = tokenizer.tokenize(text_book)
    input_ids_book = tokenizer(text_book, return_tensors='pt').input_ids[0]

Token indices sequence length is longer than the specified maximum sequence length for this model (91920 > 1024). Running this sequence through the model will result in indexing errors


In [5]:
len(input_ids_book)

91920

In [51]:
idx_layer = 5

In [52]:
predictor = nn.Sequential(
    nn.Linear(768, 1024),
    nn.GELU(),
    nn.Linear(1024, 1024),
    nn.GELU(),
    nn.Linear(1024, 1024),
    nn.GELU(),
    nn.Linear(1024, 1024),
)
                          
predictor = predictor.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(predictor.parameters(), lr=1e-2)

for idx_start in range(0, len(input_ids_book)-1024, 1024):
    idx = torch.arange(1024)+idx_start
    # # idx = (1024*torch.arange(50))[:, None] + idx
    input_ids = input_ids_book[idx].to(device)
    ak = {'debug': True}
    model(input_ids, ak=ak)
    Q, K, V = ak['QKV']
    Q = Q.permute(0, 1, 3, 2, 4).reshape(-1, 6, 1024, 64*12)[:, idx_layer]
    
    y = predictor(Q)
    yp = torch.arange(1024)[None, :].to(device)
    loss = loss_fn.forward(y, yp)
    
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    print(loss.item())

6.931952476501465
6.80461311340332
8.953631401062012
18.184497833251953
6.042055130004883
5.946381568908691
8.287891387939453
6.885945796966553
7.439345359802246
6.579103469848633
6.082755088806152
5.4809112548828125
5.181005954742432
4.827835559844971
4.783973693847656
4.627377510070801
4.498370170593262
4.453348636627197
4.467597961425781
4.302306175231934
4.259462356567383
4.141630172729492
4.131664276123047
4.03169059753418
4.038936614990234
3.9035935401916504
3.886910915374756
3.953803539276123
3.87528133392334
3.902590036392212
3.8031630516052246
3.6196298599243164
3.649967670440674
3.570722818374634
3.818276882171631
3.9187307357788086
3.9427261352539062
3.7300281524658203
3.5436630249023438
3.8783302307128906
3.7880985736846924
3.9288463592529297
3.985140085220337
3.5719099044799805
3.978423595428467
3.5593481063842773
3.765357494354248
3.6991231441497803
3.637214183807373
3.54717755317688
3.5469071865081787
3.486161708831787
3.740703582763672
3.4008171558380127
3.4325523376464